# Import & load df

In [90]:
import pandas as pd
import html

import re

In [91]:
pd.set_option('display.max_columns', None)
art_df = pd.read_csv("../data/WikiArt-Emotions/WikiArt-Emotions-All.tsv", sep='\t')

# Apply functions from 01.Image_Extraction notebook

In [92]:
%run 01_Image-Extraction.ipynb

In [93]:
art_df = clean_html_text(art_df)

In [94]:
art_df = clean_category(art_df)

# Clean years

In [95]:
def century_clean(df):
    """
    this function performs the cleanup of the values in the "Year" column. 
    It uses a dictionary to map each abbreviated century to a numeric value 
    corresponding to the average between the beginning and the approximate 
    end of the century.
    """
    
    cambios = {
        'XVI cent.': '1550',
        'XV-XVI cent.': '1550',
        'XVII cent.': '1650',
        'XVI-XVII cent.': '1600',
        'XVIII cent.': '1750',
        'XVIII-XIX cent.': '1800',
        'XIX cent.': '1850',
        'XIX-XX cent.': '1950',
        'XVII-XVIII cent.': '1750',
        'XX-XXI cent.': '2000',
        'XV cent.': '1450',
        'XIV-XV cent.': '1350',
        'XX cent.': '1900'
}
    
    df['Year'] = df['Year'].replace(cambios)
    
    return df

In [96]:
art_df = century_clean(art_df)
art_df

,ID,Style,Category,Artist,Title,Year,Is painting,Face/body,Ave. art rating,Art (image+title): agreeableness,Art (image+title): anger,Art (image+title): anticipation,Art (image+title): arrogance,Art (image+title): disagreeableness,Art (image+title): disgust,Art (image+title): fear,Art (image+title): gratitude,Art (image+title): happiness,Art (image+title): humility,Art (image+title): love,Art (image+title): optimism,Art (image+title): pessimism,Art (image+title): regret,Art (image+title): sadness,Art (image+title): shame,Art (image+title): shyness,Art (image+title): surprise,Art (image+title): trust,Art (image+title): neutral,ImageOnly: agreeableness,ImageOnly: anger,ImageOnly: anticipation,ImageOnly: arrogance,ImageOnly: disagreeableness,ImageOnly: disgust,ImageOnly: fear,ImageOnly: gratitude,ImageOnly: happiness,ImageOnly: humility,ImageOnly: love,ImageOnly: optimism,ImageOnly: pessimism,ImageOnly: regret,ImageOnly: sadness,ImageOnly: shame,ImageOnly: shyness,ImageOnly: surprise,ImageOnly: trust,ImageOnly: neutral,TitleOnly: agreeableness,TitleOnly: anger,TitleOnly: anticipation,TitleOnly: arrogance,TitleOnly: disagreeableness,TitleOnly: disgust,TitleOnly: fear,TitleOnly: gratitude,TitleOnly: happiness,TitleOnly: humility,TitleOnly: love,TitleOnly: optimism,TitleOnly: pessimism,TitleOnly: regret,TitleOnly: sadness,TitleOnly: shame,TitleOnly: shyness,TitleOnly: surprise,TitleOnly: trust,TitleOnly: neutral
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,yes,face,2.33,0.036,0.012,0.071,0.012,0.012,0.000,0.036,0.131,0.750,0.357,0.357,0.333,0.048,0.024,0.131,0.000,0.012,0.048,0.274,0.000,0.060,0.012,0.071,0.024,0.012,0.000,0.012,0.119,0.726,0.369,0.250,0.274,0.012,0.000,0.131,0.000,0.024,0.024,0.250,0.0,0.060,0.012,0.179,0.000,0.012,0.012,0.000,0.107,0.714,0.190,0.155,0.238,0.024,0.012,0.024,0.012,0.000,0.048,0.155,0.000
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,yes,body,0.70,0.000,0.000,0.500,0.000,0.400,0.100,0.300,0.000,0.100,0.000,0.100,0.100,0.200,0.100,0.100,0.000,0.000,0.500,0.100,0.000,0.000,0.000,0.100,0.000,0.000,0.100,0.300,0.000,0.100,0.100,0.000,0.200,0.200,0.100,0.200,0.000,0.000,0.500,0.000,0.0,0.000,0.000,0.600,0.000,0.300,0.000,0.200,0.000,0.300,0.000,0.200,0.000,0.100,0.000,0.000,0.000,0.000,0.200,0.000,0.000
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,yes,face,1.60,0.000,0.000,0.200,0.000,0.000,0.000,0.000,0.000,0.500,0.500,0.000,0.300,0.000,0.000,0.000,0.000,0.000,0.100,0.300,0.000,0.000,0.000,0.200,0.000,0.000,0.000,0.000,0.100,0.500,0.300,0.000,0.300,0.000,0.000,0.000,0.000,0.000,0.100,0.200,0.0,0.000,0.000,0.600,0.000,0.000,0.000,0.000,0.000,0.200,0.400,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.100,0.100,0.000
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play 'Mazeppa',1920,yes,face,0.82,0.000,0.000,0.091,0.000,0.000,0.091,0.091,0.000,0.091,0.182,0.000,0.091,0.091,0.091,0.273,0.000,0.091,0.182,0.000,0.000,0.000,0.000,0.091,0.000,0.000,0.091,0.000,0.000,0.091,0.091,0.000,0.000,0.182,0.000,0.364,0.000,0.000,0.273,0.000,0.0,0.000,0.000,0.273,0.000,0.000,0.000,0.000,0.000,0.091,0.364,0.091,0.091,0.000,0.000,0.091,0.000,0.091,0.182,0.091,0.000
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Saragoça,1828,yes,face,1.69,0.077,0.231,0.154,0.231,0.308,0.231,0.308,0.154,0.154,0.077,0.077,0.231,0.154,0.154,0.231,0.154,0.077,0.077,0.538,0.000,0.077,0.077,0.077,0.231,0.308,0.308,0.308,0.231,0.154,0.077,0.077,0.231,0.231,0.077,0.231,0.154,0.077,0.154,0.231,0.0,0.154,0.308,0.231,0.077,0.308,0.077,0.231,0.077,0.077,0.154,0.077,0.231,0.077,0.077,0.154,0.077,0.077,0.154,0.385,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [97]:
def year_clean(df):
    """
    this function performs a cleanup of the Year column. 
    It converts ranges of years to a single value as the mean rounded up.
    """
    value = df["Year"]
    try:
        start_year, end_year = map(int, value.split('-'))
        average = (start_year + end_year) // 2 + (start_year + end_year) % 2
        return average
    
    except (ValueError, AttributeError):
        return value

In [109]:
def apply_year_clean(df):
    """
    This function applies the year_clean function to the entire df to obtain
    a df with a new column containing the cleaned values of the "Year" column.
    """
    df["Year_2"] = df.apply(year_clean, axis=1)
    return df

In [110]:
art_df.Year.unique()

array(['1889', '1984', '1906', '1920', '1828', '1909', '1863', '1857',
       '1969', '1970', '1778', '1968', '1976', '1912', '1435', '1807',
       '1958', '1907', '1955', '1964', '1618', '1855', '1922', '1961',
       '1974', '1900', '1938', '1636', '1800', '1827', '1916', '1868',
       '1767', '1481-1482', '1953', '1908', '1948', '1949', '1936',
       '1991', '1450', '1927', '1890', '1998', '1850-1853', '1986',
       '1782', '1870', '1550', '1443', '1892', '1911', '1864', '1866',
       '1575', '1510', '1490-1506', '1915', '1895', '1811', '1765',
       '1785', '1763-1768', '1954', '1981', '1937', '1493', '1957',
       '1885', '1913', '1933', '1865', '1568', '1935', '1622', '1815',
       '1995', '1963', '1884-1885', '1950', '1638', '1917', '1904',
       '1975', '1525', '1880', '1850', '1607', '1797', '1960', '1966',
       '1614', '2007', '1871', '1799', '1513', '1962', '1982', '1959',
       '1791', '1800-1801', '1964-1965', '1873-1874', '1627', '1788',
       '1830', '2006',

In [108]:
art_df = apply_year_clean(art_df)
art_df

,ID,Style,Movement,Artist,Title,Year,Year_2,Painting_Y_N,Face-body,Avg_rating,IT_gratitude,IT_happiness,IT_humility,IT_love,IT_optimism,IT_trust,IT_agreeableness,IT_anticipation,IT_disagreeableness,IT_shyness,IT_surprise,IT_neutral,IT_anger,IT_arrogance,IT_disgust,IT_fear,IT_pessimism,IT_regret,IT_sadness,IT_shame,I_gratitude,I_happiness,I_humility,I_love,I_optimism,I_trust,I_agreeableness,I_anticipation,I_disagreeableness,I_shyness,I_surprise,I_neutral,I_anger,I_arrogance,I_disgust,I_fear,I_pessimism,I_regret,I_sadness,I_shame,T_gratitude,T_happiness,T_humility,T_love,T_optimism,T_trust,T_agreeableness,T_anticipation,T_disagreeableness,T_shyness,T_surprise,T_neutral,T_anger,T_arrogance,T_disgust,T_fear,T_pessimism,T_regret,T_sadness,T_shame
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,1889,yes,face,2.33,0.131,0.750,0.357,0.357,0.333,0.274,0.036,0.071,0.012,0.012,0.048,0.000,0.012,0.012,0.000,0.036,0.048,0.024,0.131,0.000,0.119,0.726,0.369,0.250,0.274,0.250,0.060,0.071,0.012,0.024,0.024,0.0,0.012,0.024,0.000,0.012,0.012,0.000,0.131,0.000,0.107,0.714,0.190,0.155,0.238,0.155,0.060,0.179,0.012,0.000,0.048,0.000,0.012,0.000,0.012,0.000,0.024,0.012,0.024,0.012
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,1984,yes,body,0.70,0.000,0.100,0.000,0.100,0.100,0.100,0.000,0.500,0.400,0.000,0.500,0.000,0.000,0.000,0.100,0.300,0.200,0.100,0.100,0.000,0.000,0.100,0.100,0.000,0.200,0.000,0.000,0.100,0.000,0.000,0.500,0.0,0.000,0.000,0.100,0.300,0.200,0.100,0.200,0.000,0.000,0.300,0.000,0.200,0.000,0.000,0.000,0.600,0.300,0.000,0.200,0.000,0.000,0.000,0.000,0.200,0.100,0.000,0.000,0.000
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,1906,yes,face,1.60,0.000,0.500,0.500,0.000,0.300,0.300,0.000,0.200,0.000,0.000,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.100,0.500,0.300,0.000,0.300,0.200,0.000,0.200,0.000,0.000,0.100,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.200,0.400,0.000,0.000,0.100,0.000,0.600,0.000,0.000,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play 'Mazeppa',1920,1920,yes,face,0.82,0.000,0.091,0.182,0.000,0.091,0.000,0.000,0.091,0.000,0.091,0.182,0.000,0.000,0.000,0.091,0.091,0.091,0.091,0.273,0.000,0.000,0.091,0.091,0.000,0.000,0.000,0.000,0.091,0.000,0.000,0.273,0.0,0.000,0.000,0.091,0.000,0.182,0.000,0.364,0.000,0.000,0.091,0.364,0.091,0.091,0.091,0.000,0.273,0.000,0.091,0.182,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.091,0.000
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Saragoça,1828,1828,yes,face,1.69,0.154,0.154,0.077,0.077,0.231,0.538,0.077,0.154,0.308,0.077,0.077,0.000,0.231,0.231,0.231,0.308,0.154,0.154,0.231,0.154,0.231,0.154,0.077,0.077,0.231,0.231,0.077,0.077,0.308,0.077,0.154,0.0,0.077,0.231,0.308,0.308,0.231,0.077,0.231,0.154,0.077,0.077,0.154,0.077,0.231,0.385,0.154,0.231,0.308,0.077,0.154,0.000,0.308,0.077,0.077,0.231,0.077,0.077,0.154,0.077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100,577287aeedc2cb388007fba9,Modern Art,Color-Field-Painting,Rupprecht Geiger,OE 260,1957,1957,yes,none,1.09,0.000,0.091,0.091,0.000,0.273,0.091,0.000,0.273,0.000,0.000,0.091,0.091,0.000,0.000,0.000,0.000,0.091,0.000,0.091,0.000,0.000,0.091,0.091,0.000,0.182,0.091,0.000,0.273,0.000,0.000,0.182,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.273,0.000,0.000,0.000,0.000,0.000,0.091,0.000,0.000,0.545,0.000,0.091,0.000,0.182,0.000,0.000,0.091,0.000,0.000,0.000,0.000,0.000
4101,57728001edc2cb3880efddcf,Modern Art,Surrealism,Oscar Dominguez,Máquina de

In [100]:
def rename__reorder_columns(df):
    """
    This function renames and orders the columns of the df.
    """
    new_column_names = {
        'ID': 'ID',
        'Style': 'Style',
        'Category': 'Movement',
        'Artist': 'Artist',
        'Title': 'Title',
        'Year': 'Year',
        'Year_2': 'Year_2',
        'Is painting': 'Painting_Y_N',
        'Face/body': 'Face-body',
        'Ave. art rating': 'Avg_rating',
        'Art (image+title): agreeableness': 'IT_agreeableness',
        'Art (image+title): anger': 'IT_anger',
        'Art (image+title): anticipation': 'IT_anticipation',
        'Art (image+title): arrogance': 'IT_arrogance',
        'Art (image+title): disagreeableness': 'IT_disagreeableness',
        'Art (image+title): disgust': 'IT_disgust',
        'Art (image+title): fear': 'IT_fear',
        'Art (image+title): gratitude': 'IT_gratitude',
        'Art (image+title): happiness': 'IT_happiness',
        'Art (image+title): humility': 'IT_humility',
        'Art (image+title): love': 'IT_love',
        'Art (image+title): optimism': 'IT_optimism',
        'Art (image+title): pessimism': 'IT_pessimism',
        'Art (image+title): regret': 'IT_regret',
        'Art (image+title): sadness': 'IT_sadness',
        'Art (image+title): shame': 'IT_shame',
        'Art (image+title): shyness': 'IT_shyness',
        'Art (image+title): surprise': 'IT_surprise',
        'Art (image+title): trust': 'IT_trust',
        'Art (image+title): neutral': 'IT_neutral',
        'ImageOnly: agreeableness': 'I_agreeableness',
        'ImageOnly: anger': 'I_anger',
        'ImageOnly: anticipation': 'I_anticipation',
        'ImageOnly: arrogance': 'I_arrogance',
        'ImageOnly: disagreeableness': 'I_disagreeableness',
        'ImageOnly: disgust': 'I_disgust',
        'ImageOnly: fear': 'I_fear',
        'ImageOnly: gratitude': 'I_gratitude',
        'ImageOnly: happiness': 'I_happiness',
        'ImageOnly: humility': 'I_humility',
        'ImageOnly: love': 'I_love',
        'ImageOnly: optimism': 'I_optimism',
        'ImageOnly: pessimism': 'I_pessimism',
        'ImageOnly: regret': 'I_regret',
        'ImageOnly: sadness': 'I_sadness',
        'ImageOnly: shame': 'I_shame',
        'ImageOnly: shyness': 'I_shyness',
        'ImageOnly: surprise': 'I_surprise',
        'ImageOnly: trust': 'I_trust',
        'ImageOnly: neutral': 'I_neutral',
        'TitleOnly: agreeableness': 'T_agreeableness',
        'TitleOnly: anger': 'T_anger',
        'TitleOnly: anticipation': 'T_anticipation',
        'TitleOnly: arrogance': 'T_arrogance',
        'TitleOnly: disagreeableness': 'T_disagreeableness',
        'TitleOnly: disgust': 'T_disgust',
        'TitleOnly: fear': 'T_fear',
        'TitleOnly: gratitude': 'T_gratitude',
        'TitleOnly: happiness': 'T_happiness',
        'TitleOnly: humility': 'T_humility',
        'TitleOnly: love': 'T_love',
        'TitleOnly: optimism': 'T_optimism',
        'TitleOnly: pessimism': 'T_pessimism',
        'TitleOnly: regret': 'T_regret',
        'TitleOnly: sadness': 'T_sadness',
        'TitleOnly: shame': 'T_shame',
        'TitleOnly: shyness': 'T_shyness',
        'TitleOnly: surprise': 'T_surprise',
        'TitleOnly: trust': 'T_trust',
        'TitleOnly: neutral': 'T_neutral'
    }

    column_order = [
        'ID',
        'Style',
        'Movement',
        'Artist',
        'Title',
        'Year',
        'Year_2',
        'Painting_Y_N',
        'Face-body',
        'Avg_rating',        
        # Image + Title
            # positive
        'IT_gratitude',
        'IT_happiness',
        'IT_humility',
        'IT_love',
        'IT_optimism',
        'IT_trust',        
            # neutral             
        'IT_agreeableness',
        'IT_anticipation',
        'IT_disagreeableness',
        'IT_shyness',
        'IT_surprise',
        'IT_neutral',
            # negative        
        'IT_anger',
        'IT_arrogance',        
        'IT_disgust',
        'IT_fear',
        'IT_pessimism',
        'IT_regret',
        'IT_sadness',
        'IT_shame',
        
        # Image
            # positive        
        'I_gratitude',
        'I_happiness',
        'I_humility',
        'I_love',
        'I_optimism',
        'I_trust',        
             # neutral                  
        'I_agreeableness',
        'I_anticipation',
        'I_disagreeableness',
        'I_shyness',
        'I_surprise',
        'I_neutral',
            # negative              
        'I_anger',
        'I_arrogance',        
        'I_disgust',
        'I_fear',
        'I_pessimism',
        'I_regret',
        'I_sadness',
        'I_shame',    
        
        # Title
            # positive        
        'T_gratitude',
        'T_happiness',
        'T_humility',
        'T_love',
        'T_optimism',
        'T_trust',        
             # neutral                  
        'T_agreeableness',
        'T_anticipation',
        'T_disagreeableness',
        'T_shyness',
        'T_surprise',
        'T_neutral',
            # negative              
        'T_anger',
        'T_arrogance',        
        'T_disgust',
        'T_fear',
        'T_pessimism',
        'T_regret',
        'T_sadness',
        'T_shame',  
            
    ]

    df = df.rename(columns=new_column_names)
    df = df[column_order]
    return df


In [101]:
art_df = rename__reorder_columns(art_df)

In [102]:
def avg_feelings(df):
    """
    This function calculates the average of the columns with positive, negative and neutral sentiments.
    """
    avg_pos = df[['IT_gratitude', 'IT_happiness', 'IT_humility', 'IT_love', 'IT_optimism', 'IT_trust']].mean(axis=1).round(3)
    df['Avg_pos'] = avg_pos

    avg_neu = df[['T_agreeableness', 'T_anticipation', 'T_disagreeableness', 'T_shyness', 'T_surprise', 'T_neutral']].mean(axis=1).round(3)
    df['Avg_neu'] = avg_neu

    avg_neg = df[['T_anger', 'T_arrogance', 'T_disgust', 'T_fear', 'T_pessimism', 'T_regret', 'T_sadness', 'T_shame']].mean(axis=1).round(3)
    df['Avg_neg'] = avg_neg

    return df

In [103]:
#art_df = avg_feelings (art_df)

In [104]:
#art_df.to_csv("../data/WikiArt-Emotions-Clean.csv", index=False)